In [1]:
import numpy as np
import cvxpy as cp
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import scipy

# Huber Regression

$$\text{minimize  }  \displaystyle\sum_{i=1}^m\phi(a_i^Tx-b_i) $$

where $\phi$ is the Huber penalty function and $a_i^T$ is the $i$th row of the given data matrix.

$$
\phi(x) = \left\{
     \begin{array}{lr}
     	u^2 &  \text{if } |u| \leq \epsilon  \\
       \epsilon(2|u|-\epsilon) & u > \epsilon
     \end{array}
   \right.
$$

for some constant $M$. The objective function is differentiable, but it is not twice continuously differentiable. So, we can use the gradient descent method, but we cannot use Newton's method. Instead, we will implement a quasi-Newton method.

We introduce a scale parameter $\sigma$ and introduce some regularization and instead solve

$$\text{minimize  }  \displaystyle\sum_{i=1}^m\left(\sigma+ \phi\left(\frac{a_i^Tx-b_i}{\sigma}\right) \sigma\right) + \gamma \|x\|_2^2 $$

In [2]:
# generate some test data
m=200
n=10
A = np.random.normal(0,1,(m,n))
real_weights = np.random.rand(10)
b = A@real_weights + np.random.normal(0,0.25, m)

## Gradient Descent

We need to compute the gradient to implement the gradient descent algorithm. 

In [3]:
def l2_norm(x):
    return np.sqrt(np.sum(x**2))
def huber(x, eps):
    return np.where(np.abs(x)<=eps, x**2, 2*np.abs(x)*eps-eps**2)
def objective(x, A ,b, sigma,eps,gamma):
    return np.sum(sigma+huber((A@x-b)/sigma,eps)*sigma) + gamma*l2_norm(x)

In [4]:
def huber_prime(x, eps):
    return np.where(np.abs(x)<=eps, 2*x, 2*eps*np.sign(x))
# last entry of vector is for variable sigma
def grad(x, A, b, sigma, eps, gamma):
    dx = A.T@huber_prime((A@x-b)/sigma, eps) + 2*gamma*x
    d_sigma = A.shape[0] + np.sum(huber((A@x-b)/sigma,eps)) - huber_prime((A@x-b)/sigma, eps)@((A@x-b)/sigma)
    return np.append(dx,d_sigma)
def neg_grad(x, A, b, sigma, eps, gamma):
    return -grad(x, A, b, sigma, eps, gamma)
# line search
def GD_backtrack(obj, x, A, b, sigma, eps, gamma, gradient, alpha, beta):
    t = 1
    # we need to make sure the scale parameter is >= 0
    while sigma-t*gradient[-1]<=0:
        t = t*beta
    while obj(x-t*gradient[:-1], A, b, sigma-t*gradient[-1], eps, gamma) > obj(x, A ,b, sigma,eps,gamma) - alpha*t*gradient@gradient:
        t = t*beta
    return t

In [5]:
# Gradient Descent
x = np.zeros(n) # initial point
sigma = 1.0
tol = 1e-5 # stopping criterion tolerance for L2 norm of gradient
max_iter = 1000
# parameters 
eps = 1.35 # huber penalty
gamma = 0.0001 # regularization
alpha = 0.1 # line search
beta = 0.5 # line search
i = 0
while i <= max_iter:
    g = grad(x, A, b, sigma, eps, gamma)
    if l2_norm(g) < tol:
        break
    else:
        t = GD_backtrack(objective, x, A, b, sigma, eps, gamma, g, alpha, beta) # step length
        x = x - t*g[:-1] # update
        sigma = sigma - t*g[-1]
        i += 1

In [6]:
x

array([0.12722972, 0.24303171, 0.86142117, 0.03919288, 0.16222664,
       0.69778093, 0.46308323, 0.11840425, 0.10479807, 0.9011498 ])

In [14]:
real_weights

array([0.12648281, 0.2602493 , 0.88087601, 0.0340526 , 0.15545873,
       0.70199606, 0.44709851, 0.13121538, 0.12740934, 0.92702782])

In [7]:
sigma

0.1672287454071263

In [8]:
hr = HuberRegressor().fit(A,b)
hr.coef_

array([0.12751572, 0.2425425 , 0.86010154, 0.03914465, 0.15856361,
       0.69529159, 0.46179127, 0.11726195, 0.10467906, 0.90067322])

In [9]:
hr.scale_

0.16983428654474925

## Quasi-Newton Method

Since the huber penalty function is not twice continuously differentiable, we cannot implement Newton's Method. Instead, we use a Quasi-Newton method. In particular, we will follow the BFGS algorithm.

In [10]:
def QN_backtrack(obj, x, A, b, sigma, eps, gamma, gradient, step, alpha, beta):
    t = 1
    # we need to make sure the scale parameter is >= 0
    while sigma+t*step[-1]<=0:
        t = t*beta
    while obj(x+t*step[:-1], A, b, sigma+t*step[-1], eps, gamma) > obj(x, A ,b, sigma,eps,gamma) + alpha*t*gradient@step:
        t = t*beta
    return t
def bfgs(H, s, y):
    return H + (y@y.T)/(y.T@s)- (H@s@s.T@H)/(s.T@(H@s))

In [11]:
# Quasi-Netwon Method
x = np.zeros(n) # initial point
sigma = 1.0
H = np.eye(n+1)
tol = 1e-5 
max_iter = 100
# parameters 
eps = 1.35 # huber penalty
gamma = 0.0001 # regularization
alpha = 0.1 # line search
beta = 0.5 # line search
s, y = np.append(x,sigma), grad(x, A, b, sigma, eps, gamma)
g = grad(x, A, b, sigma, eps, gamma)
i = 0
diff = np.inf
while i <= max_iter: # max number of iterations as stopping criterion
    if diff < tol: # stop when the difference between consecutive objective values is "small"
        break
    else:
        diff = objective(x, A ,b, sigma,eps,gamma)
        # quasi-newton direction
        qn_step = -np.linalg.inv(H)@g
        # line search for step length
        t = QN_backtrack(objective, x, A, b, sigma, eps, gamma, g, qn_step, alpha, beta)
        # update
        x = x+t*qn_step[:-1]
        sigma = sigma +t*qn_step[-1]
        # set up for bfgs update of Hessian
        diff = np.abs(objective(x, A ,b, sigma,eps,gamma) - diff)
        g = grad(x, A, b, sigma, eps, gamma)
        w = np.append(x,sigma)
        x_diff = w - s
        grad_diff = g - y
        grad_diff = grad_diff.reshape(n+1,1)
        x_diff = x_diff.reshape(n+1,1)
        H = bfgs(H,x_diff,grad_diff)
        s = w
        y = g
        i += 1

In [12]:
x

array([0.17841387, 0.15405669, 0.68692868, 0.15767734, 0.20393744,
       0.63761634, 0.37310293, 0.16572161, 0.09275366, 0.86147259])

In [13]:
sigma

3.0511776667694443e-08